In [81]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

# Load the data
Historical data for the dutch stock market is loaded.

In [82]:
AEX = pd.read_csv('AEX.csv')
date_format = '%Y-%m-%d'
AEX.Date = [datetime.strptime(date, date_format) for date in AEX.Date]
AEX['day_diff'] = AEX.Close - AEX.Open
AEX['winloss'] = abs(AEX.day_diff) / AEX.Open
AEX

,Date,Open,High,Low,Close,Adj Close,Volume,day_diff,winloss
0,1992-10-12,126.382904,127.181557,126.056175,126.945595,126.945595,0.0,0.562691,0.004452
1,1992-10-13,126.773155,127.004585,126.491806,126.850296,126.850296,0.0,0.077141,0.000608
2,1992-10-14,126.056175,126.555336,126.001724,126.487274,126.487274,0.0,0.431099,0.003420
3,1992-10-15,126.097015,126.568954,125.688614,125.824753,125.824753,0.0,-0.272262,0.002159
4,1992-10-16,126.264915,126.478195,124.858192,125.230293,125.230293,0.0,-1.034622,0.008194
...,...,...,...,...,...,...,...,...,...
7082,2020-03-11,493.369995,497.660004,482.190002,484.160004,484.160004,0.0,-9.209991,0.018668
7083,2020-03-12,460.950012,463.540009,428.089996,432.100006,432.100006,0.0,-28.850006,0.062588
7084,2020-03-13,451.760010,466.839996,425.220001,432.850006,432.850006,0.0,-18.910004,0.041859
7085,2020-03-16,409.049988,421.359985,389.600006,416.769989,416.769989,266216600.0,7.720001,0.018873


# Functions used

In [264]:
#lets make it more interactive with plotly
def plot_period_stocks(AEX, start_date, end_date, option_list, title):
    """plots the stocks for a given time period"""
    AEX = AEX.loc[AEX['Date'] >= datetime.strptime(start_date, date_format)]
    AEX = AEX.loc[AEX['Date'] <= datetime.strptime(end_date, date_format)]
    traces_list = []

    for option in option_list:
        trace = go.Scatter(x=AEX.Date,
                y=AEX[option],
                name = option,
                )
        traces_list.append(trace)

    Layout = go.Layout(xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                       yaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                       xaxis_title="Date",
                       yaxis_title="Value",
                       title=title,
                        )

    fig = go.Figure(data=traces_list, layout=Layout)
    return fig, AEX

def plot_line_in_fig(fig, AEX_s, date, text, shift=0):
    date_dt = datetime.strptime(date, date_format)
    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=date_dt,
            y0=(AEX_s.Low).min()+shift,
            x1=date_dt,
            y1=(AEX.loc[AEX['Date'] == date_dt].Close).values[0],
            line=dict(
                color="Red",
                width=1
            )
    ))
    fig.add_trace(go.Scatter(
        x=[date_dt],
        y=[(AEX_s.Low).min()+shift-15],
        text=[text],
        mode="text",
        showlegend=False,
    ))
    return fig

def plot_shape_in_fig(fig, start_date, end_date, color='green', low=350, high=575):
    from_x = datetime.strptime(start_date, date_format)
    to_x = datetime.strptime(end_date, date_format)
    fig = fig.add_shape(
        # Rectangle reference to the plot
            type="rect",
            xref="x",
            yref="y",
            x0=from_x,
            y0=low,
            x1=to_x,
            y1=high,
            line=dict(
                width=0,
            ),
            fillcolor="green",
            opacity=0.2,
        )
    return fig

# General Overview

In [205]:
option_list = ['Close', 'Open', 'High', 'Low']
title = 'Overview of AEX for past 8 years'
fig, AEX_s = plot_period_stocks(AEX, AEX.Date.min().__format__(date_format), AEX.Date.max().__format__(date_format), option_list, title)
fig = plot_line_in_fig(fig, AEX_s, '1997-07-02', 'Asian crisis',40)
fig = plot_line_in_fig(fig, AEX_s, '1998-08-17', 'Russian Recession')
fig = plot_line_in_fig(fig, AEX_s, '2000-04-14', 'Dotcom', 40)
fig = plot_line_in_fig(fig, AEX_s, '2001-09-10', '9/11', 80)
fig = plot_line_in_fig(fig, AEX_s, '2008-01-02', 'Market Crash 2008')
fig = plot_line_in_fig(fig, AEX_s, '2010-04-27', 'Credit Crisis', 40)
fig = plot_line_in_fig(fig, AEX_s, '2015-06-12', 'China Stock Crash', 40)
fig = plot_line_in_fig(fig, AEX_s, '2018-09-20', 'Stock Market Downturn')
fig = plot_line_in_fig(fig, AEX_s, '2020-02-18', 'Covid-19', 40)
fig

Looking at these effects, the Dotcom in 2001 and the 2008 market crash had the higest impact on the AEX next to the Covid-19. Because of this we will make an analysis of these crashes (financially forced) and compare them to the Covid-19 situation.

# Analysis of stock crash
We compare the graph for the crash with 2001 and 2008 for the dot.com bubble and the 2008 market crash

In [281]:
##### For Covid-19
#### check AEX Stocks from 01-01-2020 till today.

## options are: 'Close', 'Open', 'High', 'Low', 'day_diff', 'winloss'
option_list = ['Close', 'Open', 'High', 'Low']
title = 'COVID-19'
fig, AEX_s = plot_period_stocks(AEX, '2020-01-01', datetime.today().__format__(date_format), option_list, title)
fig = plot_line_in_fig(fig, AEX_s, '2020-01-24', 'First case EU')
fig = plot_line_in_fig(fig, AEX_s, '2020-02-28', 'First case NL')
fig = plot_shape_in_fig(fig, '2020-01-29', '2020-02-06', high=650)
fig = plot_shape_in_fig(fig, '2020-02-10', '2020-02-24', high=650)
fig = plot_shape_in_fig(fig, '2020-02-28', '2020-03-06', high=650)
fig = plot_shape_in_fig(fig, '2020-03-12', '2020-03-16', high=650)

fig

In [275]:
## For comparisation, stock crash of 2008
## options are: 'Close', 'Open', 'High', 'Low', 'day_diff', 'winloss'
option_list = ['Close', 'Open', 'High', 'Low']
title = 'The stock market crash in 2008'
fig, AEX_s = plot_period_stocks(AEX, '2007-12-01', '2008-02-18', option_list, title)
fig = plot_line_in_fig(fig, AEX_s, '2008-01-02', 'Market Crash 2008')
fig = plot_shape_in_fig(fig, '2007-12-12', '2007-12-21')
fig = plot_shape_in_fig(fig, '2007-12-22', '2008-01-03')
fig = plot_shape_in_fig(fig, '2008-01-10', '2008-01-15')
fig = plot_shape_in_fig(fig, '2008-01-21', '2008-01-25')

fig

In [272]:
## For comparisation, stock crash of 2008
## options are: 'Close', 'Open', 'High', 'Low', 'day_diff', 'winloss'
option_list = ['Close', 'Open', 'High', 'Low']
title = 'The Dotcom bubble 2001 - highest drop in 2001'
fig, AEX_s = plot_period_stocks(AEX, '2001-08-01', '2001-10-15', option_list, title)
fig = plot_line_in_fig(fig, AEX_s, '2001-08-10', 'Dotcom bubble 2001')
fig = plot_shape_in_fig(fig, '2001-08-16', '2001-08-23')
fig = plot_shape_in_fig(fig, '2001-08-24', '2001-08-31')
fig = plot_shape_in_fig(fig, '2001-09-09', '2001-09-14')
fig = plot_shape_in_fig(fig, '2001-09-20', '2001-09-24')


fig

### Re-occurring themes
All the graphs are of a same time period and show similar trends:
- There is a period of slight drop in stocks at the beginning
- Then a small rise in overal stocks
- then stocks start to drop, but there is an seemfull recovery in the middle
- then the drop steeply drops to 400 or less.

# Comparing to other outbreaks

In [216]:
## For comparisation, stock crash of 2008
## options are: 'Close', 'Open', 'High', 'Low', 'day_diff', 'winloss'
option_list = ['Close', 'Open', 'High', 'Low']
title = 'during the 2002 /2003 /2004 SARS outbreak'
fig, AEX_s = plot_period_stocks(AEX, '2002-11-01', '2004-05-01', option_list, title)

# add label and vertical for the sars outbreak date
fig = plot_line_in_fig(fig, AEX_s, '2002-11-18', 'SARS')
fig

In [284]:
## For comparisation, stock crash of 2008
## options are: 'Close', 'Open', 'High', 'Low', 'day_diff', 'winloss'
option_list = ['Close', 'Open', 'High', 'Low']
title = 'Pneunomic Plague'
fig, AEX_s = plot_period_stocks(AEX, '1994-08-01', '1994-12-01', option_list, title)

# add label and vertical for the sars outbreak date
fig = plot_line_in_fig(fig, AEX_s, '1994-09-01', 'Pneumonic Plague')
fig

# more stuff to think about:

In [217]:
option_list = ['day_diff']
title=' Daily difference in value'
fig, AEX_s = plot_period_stocks(AEX, '2020-01-02', datetime.today().__format__(date_format), option_list, title)
fig

In [74]:
option_list = ['winloss']
title = 'Percentage of loss'
fig, AEX_s = plot_period_stocks(AEX, '2020-01-01', datetime.today().__format__(date_format), option_list, title)
fig